# Breast Cancer prediction machine learning algorithm using logistic regression


### The data was obtained from https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Original)
### Missing values with "?" were replaced with "NaN" to make it easy to find and delete rows with missing data
### Values in the field classs was replaced as follows 2=0(benign) 4=1 (malignant /cancer)
### No other pre-processing was done

### Data description can be found at https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names

### We are going to use breast cancer cell cytology features ( clump thickness, uniformity of cell size, uniformity of cell shape, marginal adhesion, single epithelial cell size, bare nuclei, bland chromatin, normal nuceloli, mitosis) to predict the class (benign = 0, malignant /cancer = 1)

In [2]:
# Importing pandas to read CSV file

In [3]:
import pandas as pd

In [4]:
# Read breast cancer csv file
data = pd.read_csv('wisconsin_breast_cancer.csv')

In [5]:
# Display the first 5 rows
data.head()

,id,thickness,size,shape,adhesion,single,nuclei,chromatin,nucleoli,mitosis,class
0,1000025,5,1,1,1,2,1.0,3,1,1,0
1,1002945,5,4,4,5,7,10.0,3,2,1,0
2,1015425,3,1,1,1,2,2.0,3,1,1,0
3,1016277,6,8,8,1,3,4.0,3,7,1,0
4,1017023,4,1,1,3,2,1.0,3,1,1,0


In [6]:
data.shape # It is always a good idea to understand your data

(699, 11)

In [7]:
# There are 699 rows and 11 columns in this CSV file

# Delete rows with missing data

In [8]:
data.isnull().sum() # Find out how many cells have missing values
# Looks like the field nucleoli has 16 missing values 

id            0
thickness     0
size          0
shape         0
adhesion      0
single        0
nuclei       16
chromatin     0
nucleoli      0
mitosis       0
class         0
dtype: int64

In [9]:
data=data.dropna(how='any') # Dropping any rows that has missing values

In [10]:
# previously we had 699 rows. The above code deleted 16 rows with missing values and now we have 683 rows

In [11]:
#creating feature data set
# Here we are going to select feature fileds and assign it to a pandas data frame x. We do not need Id. So it was dropped
x=data[['thickness','size','shape','adhesion','single','nuclei','chromatin','nucleoli','mitosis']] 

In [12]:
x.head() # printing the first 5 rows to see whether we got all the features 

,thickness,size,shape,adhesion,single,nuclei,chromatin,nucleoli,mitosis
0,5,1,1,1,2,1.0,3,1,1
1,5,4,4,5,7,10.0,3,2,1
2,3,1,1,1,2,2.0,3,1,1
3,6,8,8,1,3,4.0,3,7,1
4,4,1,1,3,2,1.0,3,1,1


In [13]:
# Now we are going to extract the 'class' field from 'data' and store it in variable y
y=data['class']
y.head()

0    0
1    0
2    0
3    0
4    0
Name: class, dtype: int64

# Getting ready to do classification
### Import cross validation from sklearn for test trian split
### then import logistic regression from sklearn


In [14]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [15]:
# train logistic regression model 
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
y_pred_class=logreg.predict(x_test) # make predictions based on x_test and store it to y_pred_class

In [17]:
# Now let us see how our model is performing. 
# We will start with accuracy
from sklearn import metrics
print metrics.accuracy_score(y_test, y_pred_class)

0.929824561404


In [18]:
## Not bad at all. 92.9% of the time our model was able to identify breast cancer based on the training data 
# Now let us see how our "intelligent" model compare to a dumb model which predicts "o" or "not cancer" all the time
# because it the most prevelent class

In [19]:
y_test.value_counts() # as you see "0" is more prevalent 

0    107
1     64
Name: class, dtype: int64

In [20]:
# Now let us see the percentage of "o" in y_test. This will be the accuracy of a dumb model that predicts 0 
# all the time
1-y_test.mean()

0.6257309941520468

In [21]:
# looks like we did better than the dumb model!

# Now let us create a confusion matrix to identify sensitivity specificity & all  the other good statistical stuff

In [22]:
print metrics.confusion_matrix(y_test, y_pred_class)

[[103   4]
 [  8  56]]


In [23]:
#  let us see what this means
#
#                Predicted 0    Predicted 1    Total
#                                
#Actual  0        103              4            107
#Actual  1          8             56             64           
#Total            111             60 

In [24]:
confusion =metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [25]:
# Let us see the sensitivity of our logistic regression model
print TP / float(TP+FN)

0.875


In [26]:
# Our model's sensitivity is 87.5%

In [27]:
# Let us calculate specificity
print TN / float(TN+FP)

0.96261682243


In [28]:
# Looks like our model has pretty good specificity 96.2%

In [29]:
# Calculate false postive rate - predicting cancer when pt does not have cancer
print FP/ float(TN+FP)

0.0373831775701


In [30]:
# pretty awesome 3.7%

In [31]:
# precison - when it is predicting cancer how precise is it 
# positive predictive value 
print TP / float(TP+FP)

0.933333333333


In [32]:
# 93.3% of the time 

In [33]:
# Negative predictive value
print TN / float(TN+ FN)

0.927927927928


In [34]:
# Now let us say that you want to figure out the probaility of cancer from a set of features
# we can use the predict_proba function
# let us first see the predicted answers
logreg.predict(x_test)[0:10] # predicting cancer  based on the data from first 10 rows of x


array([0, 0, 1, 1, 0, 0, 0, 1, 0, 0])

In [35]:
# Now let us predict the probaility of each prediction
logreg.predict_proba(x_test)[0:10, :]

array([[ 0.92925368,  0.07074632],
       [ 0.97810202,  0.02189798],
       [ 0.05497954,  0.94502046],
       [ 0.01971146,  0.98028854],
       [ 0.98442876,  0.01557124],
       [ 0.96262046,  0.03737954],
       [ 0.9421951 ,  0.0578049 ],
       [ 0.00202679,  0.99797321],
       [ 0.9598277 ,  0.0401723 ],
       [ 0.98330704,  0.01669296]])

In [36]:
# The first colun is the probability of it being benign. Second column is the probablity of it being cancerous 

### Now let us find out the area under the curve. 
#### "roc_auc_score function computes the area under the receiver operating characteristic (ROC) curve"

In [38]:
from sklearn.metrics import roc_auc_score
# calculates the probability of predicting "1" (cancer) and store the out put in probab_cancer
proba_cancer=logreg.predict_proba(x_test)[:,1]  

In [41]:
# we need the actual values in the cancer column and the predicted probabilities of postive value "1"
roc_auc_score(y_test, proba_cancer)

0.99269859813084116

# The End